<a href="https://colab.research.google.com/github/krishnamani77/TF2_Notebooks/blob/master/char-rnn" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
import keras.utils

sh_url = 'https://homl.info/shakespeare'
sh_file = "sh.txt"
filepath = keras.utils.get_file(sh_file, sh_url)

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
with open(filepath) as f:
  sh_txt = f.read()  

In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([sh_txt])

In [8]:
print(tokenizer.texts_to_sequences(['Hello world']))
print(tokenizer.sequences_to_texts([[7,2,12,12,4], [1,17,4,9,12,13]]))

[[7, 2, 12, 12, 4, 1, 17, 4, 9, 12, 13]]
['h e l l o', '  w o r l d']


In [13]:
max_id = len(tokenizer.word_index)
print(max_id)

39


In [14]:
print(tokenizer.word_index)

{' ': 1, 'e': 2, 't': 3, 'o': 4, 'a': 5, 'i': 6, 'h': 7, 's': 8, 'r': 9, 'n': 10, '\n': 11, 'l': 12, 'd': 13, 'u': 14, 'm': 15, 'y': 16, 'w': 17, ',': 18, 'c': 19, 'f': 20, 'g': 21, 'b': 22, 'p': 23, ':': 24, 'k': 25, 'v': 26, '.': 27, "'": 28, ';': 29, '?': 30, '!': 31, '-': 32, 'j': 33, 'q': 34, 'x': 35, 'z': 36, '3': 37, '&': 38, '$': 39}


In [15]:
dataset_size = tokenizer.document_count
print(dataset_size)

1


In [0]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([sh_txt]))

In [20]:
print(encoded[10:20])

[36  2 10 24 11 22  2 20  4  9]
